In [1]:
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

In [2]:
AAD_TENANT_ID = "0929023f-428c-4eb7-a982-a21c9bae3bc7"
KUSTO_URI = "https://kvc-m23a83j31rynmvx6wh.northeurope.kusto.windows.net/"
KUSTO_INGEST_URI = "https://ingest-kvc-m23a83j31rynmvx6wh.northeurope.kusto.windows.net/"
KUSTO_DATABASE = "MyDatabase"

In [3]:
KCSB_INGEST = KustoConnectionStringBuilder.with_interactive_login(
    KUSTO_INGEST_URI)

KCSB_DATA = KustoConnectionStringBuilder.with_interactive_login(
    KUSTO_URI)

DESTINATION_TABLE = "StormEvents"
DESTINATION_TABLE_COLUMN_MAPPING = "StormEvents_CSV_Mapping"

In [8]:
from azure.kusto.data import DataFormat
from azure.kusto.ingest import QueuedIngestClient, IngestionProperties, FileDescriptor, BlobDescriptor, ReportLevel, ReportMethod

CONTAINER = "samplefiles"
ACCOUNT_NAME = "kustosamples"
SAS_TOKEN = ""  # If relevant add SAS token
FILE_PATH = "StormEvents.csv"
FILE_SIZE = 64158321    # in bytes

BLOB_PATH = "https://" + ACCOUNT_NAME + ".blob.core.windows.net/" + \
    CONTAINER + "/" + FILE_PATH + SAS_TOKEN

In [9]:
KUSTO_CLIENT = KustoClient(KCSB_DATA)
CREATE_TABLE_COMMAND = ".create table StormEvents (StartTime: datetime, EndTime: datetime, EpisodeId: int, EventId: int, State: string, EventType: string, InjuriesDirect: int, InjuriesIndirect: int, DeathsDirect: int, DeathsIndirect: int, DamageProperty: int, DamageCrops: int, Source: string, BeginLocation: string, EndLocation: string, BeginLat: real, BeginLon: real, EndLat: real, EndLon: real, EpisodeNarrative: string, EventNarrative: string, StormSummary: dynamic)"

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_TABLE_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,TableName,Schema,DatabaseName,Folder,DocString
0,StormEvents,"{""Name"":""StormEvents"",""OrderedColumns"":[{""Name...",MyDatabase,None,None


In [10]:
CREATE_MAPPING_COMMAND = """.create table StormEvents ingestion csv mapping 'StormEvents_CSV_Mapping' '[{"Name":"StartTime","datatype":"datetime","Ordinal":0}, {"Name":"EndTime","datatype":"datetime","Ordinal":1},{"Name":"EpisodeId","datatype":"int","Ordinal":2},{"Name":"EventId","datatype":"int","Ordinal":3},{"Name":"State","datatype":"string","Ordinal":4},{"Name":"EventType","datatype":"string","Ordinal":5},{"Name":"InjuriesDirect","datatype":"int","Ordinal":6},{"Name":"InjuriesIndirect","datatype":"int","Ordinal":7},{"Name":"DeathsDirect","datatype":"int","Ordinal":8},{"Name":"DeathsIndirect","datatype":"int","Ordinal":9},{"Name":"DamageProperty","datatype":"int","Ordinal":10},{"Name":"DamageCrops","datatype":"int","Ordinal":11},{"Name":"Source","datatype":"string","Ordinal":12},{"Name":"BeginLocation","datatype":"string","Ordinal":13},{"Name":"EndLocation","datatype":"string","Ordinal":14},{"Name":"BeginLat","datatype":"real","Ordinal":16},{"Name":"BeginLon","datatype":"real","Ordinal":17},{"Name":"EndLat","datatype":"real","Ordinal":18},{"Name":"EndLon","datatype":"real","Ordinal":19},{"Name":"EpisodeNarrative","datatype":"string","Ordinal":20},{"Name":"EventNarrative","datatype":"string","Ordinal":21},{"Name":"StormSummary","datatype":"dynamic","Ordinal":22}]'"""

RESPONSE = KUSTO_CLIENT.execute_mgmt(KUSTO_DATABASE, CREATE_MAPPING_COMMAND)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Name,Kind,Mapping,LastUpdatedOn,Database,Table
0,StormEvents_CSV_Mapping,Csv,"[{""Name"":""StartTime"",""DataType"":""datetime"",""Or...",2025-07-12 08:21:30.534848700+00:00,MyDatabase,StormEvents


In [11]:
INGESTION_CLIENT = QueuedIngestClient(KCSB_INGEST)

# All ingestion properties are documented here: https://learn.microsoft.com/azure/kusto/management/data-ingest#ingestion-properties
INGESTION_PROPERTIES = IngestionProperties(database=KUSTO_DATABASE, table=DESTINATION_TABLE, data_format=DataFormat.CSV,
                                           ingestion_mapping_reference=DESTINATION_TABLE_COLUMN_MAPPING, additional_properties={'ignoreFirstRecord': 'true'})
# FILE_SIZE is the raw size of the data in bytes
BLOB_DESCRIPTOR = BlobDescriptor(BLOB_PATH, FILE_SIZE)
INGESTION_CLIENT.ingest_from_blob(
    BLOB_DESCRIPTOR, ingestion_properties=INGESTION_PROPERTIES)

print('Done queuing up ingestion with Azure Data Explorer')

Done queuing up ingestion with Azure Data Explorer


In [12]:
QUERY = "StormEvents | count"

RESPONSE = KUSTO_CLIENT.execute_query(KUSTO_DATABASE, QUERY)

dataframe_from_result_table(RESPONSE.primary_results[0])

,Count
0,59066


In [16]:
show_mapping_query = ".show ingestion mappings"
res_ = KUSTO_CLIENT.execute_query(KUSTO_DATABASE, "StormEvents| limit 10")
df_ = dataframe_from_result_table(res_.primary_results[0])
print(type(df_))
df_

<class 'pandas.core.frame.DataFrame'>


,StartTime,EndTime,EpisodeId,EventId,State,EventType,InjuriesDirect,InjuriesIndirect,DeathsDirect,DeathsIndirect,...,Source,BeginLocation,EndLocation,BeginLat,BeginLon,EndLat,EndLon,EpisodeNarrative,EventNarrative,StormSummary
0,2007-01-01 00:00:00+00:00,2007-01-01 00:00:00+00:00,2592,13208,NORTH CAROLINA,Thunderstorm Wind,0,0,0,0,...,Public,CASAR,CASAR,-81.63,35.52,-81.63,<NA>,Several trees down.,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
1,2007-01-01 00:00:00+00:00,2007-01-01 05:00:00+00:00,4171,23358,WISCONSIN,Winter Storm,0,0,0,0,...,COOP Observer,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
2,2007-01-01 00:00:00+00:00,2007-01-01 05:00:00+00:00,4171,23357,WISCONSIN,Winter Storm,0,0,0,0,...,COOP Observer,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 0,\r\n ""StartTime"": ""2...",None
3,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9494,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 2000,\r\n ""StartTime"":...",None
4,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9488,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 2000,\r\n ""StartTime"":...",None
5,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9487,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 2000,\r\n ""StartTime"":...",None
6,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9485,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 3000,\r\n ""StartTime"":...",None
7,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9486,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 3000,\r\n ""StartTime"":...",None
8,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9493,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 2000,\r\n ""StartTime"":...",None
9,2007-01-01 00:00:00+00:00,2007-01-01 06:00:00+00:00,1930,9489,NEW YORK,Winter Weather,0,0,0,0,...,Department of Highways,,,<NA>,<NA>,<NA>,<NA>,,"{\r\n ""TotalDamages"": 2000,\r\n ""StartTime"":...",None
